# Akses & Install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install jsons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# TRAIN

In [4]:
# Train Data

import random
import json
import pickle
import numpy as np

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/drive/MyDrive/Semester 5/Capstone/chatbot/intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag =[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5)
model.save('chatbotmodel.h5', hist)

print('Done')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
15/15 [==============================] - 1s 2ms/step - loss: 3.1480 - accuracy: 0.0405
Epoch 2/100
15/15 [==============================] - 0s 2ms/step - loss: 3.1117 - accuracy: 0.0811
Epoch 3/100
15/15 [==============================] - 0s 2ms/step - loss: 3.0562 - accuracy: 0.1622
Epoch 4/100
15/15 [==============================] - 0s 2ms/step - loss: 2.9386 - accuracy: 0.1892
Epoch 5/100
15/15 [==============================] - 0s 2ms/step - loss: 2.9046 - accuracy: 0.1892
Epoch 6/100
15/15 [==============================] - 0s 2ms/step - loss: 2.8548 - accuracy: 0.2027
Epoch 7/100
15/15 [==============================] - 0s 2ms/step - loss: 2.7489 - accuracy: 0.2568
Epoch 8/100
15/15 [==============================] - 0s 2ms/step - loss: 2.6129 - accuracy: 0.2432
Epoch 9/100
15/15 [==============================] - 0s 2ms/step - loss: 2.5204 - accuracy: 0.2973
Epoch 10/100
15/15 [==============================] - 0s 2ms/step - loss: 2.6273 - accuracy: 0.2297
Epoch 11/

# TES

In [5]:
import random
import json
import pickle
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/drive/MyDrive/Semester 5/Capstone/chatbot/intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda  x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list,intents_json):
    tag= intents_list[0]['intent']
    list_of_intents =intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("|============= Welcome to College Equiry Chatbot System! =============|")
print("|============================== Feel Free ============================|")
print("|================================== To ===============================|")
print("|=============== Ask your any query about our college ================|")
while True:
    message = input("| You: ")
    if message == "bye" or message == "Goodbye" or message == "dadah" or message == "see you" or message == "babay" or message == "sayonara" or message == "sayonara":
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)
        exit()
        quit()
        print("|===================== The Program End here! =====================|")
        break

    else:
      ints = predict_class(message)
      res = get_response(ints, intents)
      print("| Bot:", res)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


|============= Welcome to College Equiry Chatbot System! =============|
|============================== Feel Free ============================|
|================================== To ===============================|
|=============== Ask your any query about our college ================|
| You: hi
1/1 [==============================] - 0s 86ms/step
| Bot: Hai!
| You: bye
1/1 [==============================] - 0s 19ms/step
| Bot: Goodbye
|===================== The Program End here! =====================|
